In [425]:
import gzip            
import io              
import numpy as np
import pandas as pd    
import pymysql.cursors 
import rdflib
from rdflib import Namespace
import urllib.request 
import math

import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

# Connect to the database

connection = pymysql.connect(host='hosting.nyu.edu',
                             user='cmrougha_adsq',
                             password='###REPLACE###',
                             db='cmrougha_adsq2017',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

# A Function to Convert to RDF

In [426]:
resourceKey = 'www.astronomoumenos.com/id/'
verbKey = 'www.astronomoumenos.com/ontologies/astr.owl#'

def convertToRDF(g,df,convType,class1,class2=''):
    cols = list(df)
    for t in df.iterrows():
        s = rdflib.URIRef(resourceKey + str(t[1][cols[0]]))
        
        if cols[1] == "rdfs:label":
            p = rdflib.URIRef("http://www.w3.org/2000/01/rdf-schema#label")
        else:
            p = rdflib.URIRef(verbKey + cols[1])
            
        g.add((s,rdflib.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),rdflib.URIRef('www.astronomoumenos.com/ontologies/astr.owl#' + class1)))
        
        if convType == "resource-resource":
            if str(t[1][cols[1]]).strip() != '':
                o = rdflib.URIRef(resourceKey + str(t[1][cols[1]]))
                g.add((s,p,o))
                
                g.add((o,rdflib.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),rdflib.URIRef('www.astronomoumenos.com/ontologies/astr.owl#' + class2)))
        
        if convType == "resource-string":
            if str(t[1][cols[1]]).strip() != '':
                o = rdflib.Literal(str(t[1][cols[1]]))
                g.add((s,p,o))
        
        if convType == "resource-numeric":
            pass
        
        if convType == "resource-resource-source":
            if str(t[1][cols[1]]).strip() != '':
                bn = rdflib.BNode("_bn" + str(t[1][cols[0]] + "p"))
                bp = rdflib.URIRef(verbKey + "hasClaim")
                bc = rdflib.URIRef(resourceKey + "pinakes")
            
                o = rdflib.URIRef(resourceKey + str(t[1][cols[1]]))
            
                g.add((s,bp,bn))
                g.add((bn,p,o))
                g.add((bn,rdflib.URIRef(verbKey + "claimedBy"),bc))
        
                g.add((o,rdflib.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),rdflib.URIRef('www.astronomoumenos.com/ontologies/astr.owl#' + class2)))
                g.add((bc,rdflib.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),rdflib.URIRef('www.astronomoumenos.com/ontologies/astr.owl#Catalog')))
        
        if convType == "resource-string-source":
            if str(t[1][cols[1]]).strip() != '':
                bn = rdflib.BNode("_bn" + str(t[1][cols[0]] + "p"))
                bp = rdflib.URIRef(verbKey + "hasClaim")
                bc = rdflib.URIRef(resourceKey + "pinakes")
            
                o = rdflib.Literal(str(t[1][cols[1]]))
            
                g.add((s,bp,bn))
                g.add((bn,p,o))
                g.add((bn,rdflib.URIRef(verbKey + "claimedBy"),bc))
        
                g.add((bc,rdflib.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),rdflib.URIRef('www.astronomoumenos.com/ontologies/astr.owl#Catalog')))

        
        if convType == "resource-numeric-source":
            pass
        
        if convType == "blank-blank-source":
            bn_s = rdflib.BNode("_bn" + str(t[1][cols[0]] + "p"))
            bn_p = rdflib.URIRef(verbKey + cols[1])
            bn_o = rdflib.BNode("_bn" + str(t[1][cols[1]] + "p"))
            bc = rdflib.URIRef(resourceKey + "pinakes")
            o = rdflib.URIRef(resourceKey + str(t[1][cols[1]]))
            
            g.add((s,rdflib.URIRef(verbKey + "hasClaim"),bn_s))
            g.add((bn_s,rdflib.URIRef(verbKey + "claimedBy"),bc))
            g.add((o,rdflib.URIRef(verbKey + "hasClaim"),bn_o))
            g.add((bn_o,rdflib.URIRef(verbKey + "claimedBy"),bc))
            g.add((bn_s,bn_p,bn_o))
        
    return g
    


# Converting to RDF

In [427]:
g = rdflib.Graph()

In [428]:
# Manuscript locatedIn Institution
sql = """
SELECT DISTINCT CONCAT("ms",astrID_ms) AS contains, REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Depot,'"',''),')',''),'(',''),"'",""),'.',''),',',''),' ','') AS locatedIn 
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
ms_inst = pd.DataFrame(result, columns = names)

#ms_inst.head()

g = convertToRDF(g,ms_inst,'resource-resource','ExtantManuscript','Institution')

In [429]:
# Institution locatedIn City
sql = """
SELECT DISTINCT REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Depot,'"',''),')',''),'(',''),"'",""),'.',''),',',''),' ','') AS contains, REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Ville,'"',''),')',''),'(',''),"'",""),'.',''),',',''),' ','') AS locatedIn 
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
inst_city = pd.DataFrame(result, columns = names)

#inst_city.head()

g = convertToRDF(g,inst_city,'resource-resource','Institution','City')

In [430]:
# City locatedIn Country
sql = """
SELECT DISTINCT REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Ville,'"',''),')',''),'(',''),"'",""),'.',''),',',''),' ','') AS contains, REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Pays,'"',''),')',''),'(',''),"'",""),'.',''),',',''),' ','') AS locatedIn 
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
city_country = pd.DataFrame(result, columns = names)

#city_country.head()

g = convertToRDF(g,city_country,'resource-resource','City','Country')

In [431]:
# Manuscript rdfs:label Shelfmark 
sql = """
SELECT DISTINCT CONCAT("ms",astrID_ms) AS ``, CONCAT(Ville," ",Depot," ",FondsCote) AS `rdfs:label`
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
ms_shelf = pd.DataFrame(result, columns = names)

#ms_shelf.head()

g = convertToRDF(g,ms_shelf,'resource-string','Manuscript')

In [432]:
# Manuscript msContains Witness
sql = """
SELECT DISTINCT CONCAT("ms",astrID_ms) AS inMS, CONCAT("wi",astrID_wi) AS msContains
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
ms_witness = pd.DataFrame(result, columns = names)

#ms_witness.head()

g = convertToRDF(g,ms_witness,'resource-resource','Manuscript','Witness')

In [433]:
# Witness witnessOf Work
sql = """
SELECT DISTINCT CONCAT("wi",astrID_wi) AS hasWitness, CONCAT("wo",astrID_wo) AS witnessOf
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
witness_work = pd.DataFrame(result, columns = names)

#witness_work.head()

g = convertToRDF(g,witness_work,'resource-resource','Witness','Work')

In [434]:
# Work hasContributor Contributor
sql = """
SELECT DISTINCT CONCAT("wo",astrID_wo) AS ``, CONCAT("au",astrID_au) AS hasContributor
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
work_author = pd.DataFrame(result, columns = names)

#work_author.head()

g = convertToRDF(g,work_author,'resource-resource','Work','Contributor')

In [435]:
# Work rdfs:label Title
sql = """
SELECT DISTINCT CONCAT("wo",astrID_wo) AS ``, Oeuvre AS `rdfs:label`
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
work_label = pd.DataFrame(result, columns = names)

#work_label.head()

g = convertToRDF(g,work_label,'resource-string','Work')

In [436]:
# Author rdfs:label Name
sql = """
SELECT DISTINCT CONCAT("au",astrID_au) AS ``, Auteur AS `rdfs:label`
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
author_label = pd.DataFrame(result, columns = names)

#author_label.head()

g = convertToRDF(g,author_label,'resource-string','Author')

In [437]:
# Institution rdfs:label Name
sql = """
SELECT DISTINCT REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Depot,'"',''),')',''),'(',''),"'",""),'.',''),',',''),' ','') AS ``, Depot AS `rdfs:label` 
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
inst_label = pd.DataFrame(result, columns = names)

#inst_label.head()

g = convertToRDF(g,inst_label,'resource-string','Institution')

In [438]:
# City rdfs:label Name
sql = """
SELECT DISTINCT REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Ville,'"',''),')',''),'(',''),"'",""),'.',''),',',''),' ','') AS ``, Ville AS `rdfs:label` 
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
city_label = pd.DataFrame(result, columns = names)

#city_label.head()

g = convertToRDF(g,city_label,'resource-string','City')

In [439]:
# Country rdfs:label Name
sql = """
SELECT DISTINCT REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Pays,'"',''),')',''),'(',''),"'",""),'.',''),',',''),' ','') AS ``, Pays AS `rdfs:label` 
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
country_label = pd.DataFrame(result, columns = names)

#country_label.head()

g = convertToRDF(g,country_label,'resource-string','Country')

In [440]:
# Manuscript hasClaim _bn hasID Pinakes Identifier (claimBy Pinakes)
sql = """
SELECT DISTINCT CONCAT("ms",astrID_ms) AS claimAbout, MS AS hasID
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
ms_pinID = pd.DataFrame(result, columns = names)

#ms_pinID.head()

g = convertToRDF(g,ms_pinID,'resource-string-source','Manuscript')

In [441]:
# Witness hasClaim _bn pinakesFolios Folios (claimBy Pinakes)
sql = """
SELECT DISTINCT CONCAT("wi",astrID_wi) AS claimAbout, Folios AS pinakesFolios
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
witness_folios = pd.DataFrame(result, columns = names)

#witness_folios.head()

g = convertToRDF(g,witness_folios,'resource-string-source','Witness')

In [442]:
# Witness hasClaim _bn pinakesSiecle Century (claimBy Pinakes)
sql = """
SELECT DISTINCT CONCAT("wi",astrID_wi) AS claimAbout, Siecle AS pinakesSiecle
FROM `17.4.12_pinAstr_all`
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
witness_siecle = pd.DataFrame(result, columns = names)

#witness_siecle.head()

g = convertToRDF(g,witness_siecle,'resource-string-source','Witness')

What is the most effective way of talking about the order of texts in manuscripts? A numerical order or predicates to the effect of `immediatelyPrecededBy`, `precededBy`, `immediatelyFollowedBy`, `followedBy`?

To get at the latter, we can use the following query:
```
SELECT x.astrID_wi AS immediatelyPrecededBy, y.astrID_wi AS immediatelyFollowedBy
FROM `17.4.12_pinAstr_all` AS x
 INNER JOIN `17.4.12_pinAstr_all` AS y
 ON ( x.astrID_ms = y.astrID_ms ) AND ( y.Order = x.Order + 1 )
```

In [443]:
# Witness0 hasClaim _bn0 immediatelyPrecededBy _bn1 ^hasClaim Witness1 (claimBy Pinakes)
sql = """
SELECT CONCAT("wi",x.astrID_wi) AS immediatelyPrecededBy, CONCAT("wi",y.astrID_wi) AS immediatelyFollowedBy
FROM `17.4.12_pinAstr_all` AS x
 INNER JOIN `17.4.12_pinAstr_all` AS y
 ON ( x.astrID_ms = y.astrID_ms ) AND ( y.Order = x.Order + 1 )
 """
with connection.cursor() as cursor:
    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()
witness_witness = pd.DataFrame(result, columns = names)

#witness_witness.head()

g = convertToRDF(g,witness_witness,'blank-blank-source','Witness','Witness')

.

.

.

.

.

.

.

.

.

# Sample Queries

In [444]:
result = g.query(
        """
        PREFIX astr: <www.astronomoumenos.com/ontologies/astr.owl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT * WHERE {
            ?1s ?2p ?3o .
            }""")

pd.DataFrame(result.bindings)

,1s,2p,3o
0,_bnwi5667174p,www.astronomoumenos.com/ontologies/astr.owl#cl...,www.astronomoumenos.com/id/pinakes
1,_bnms9595953p,www.astronomoumenos.com/ontologies/astr.owl#hasID,46359
2,www.astronomoumenos.com/id/wi1599367,www.astronomoumenos.com/ontologies/astr.owl#ha...,_bnwi1599367p
3,_bnwi5409022p,www.astronomoumenos.com/ontologies/astr.owl#pi...,015v-16v
4,_bnwi8234825p,www.astronomoumenos.com/ontologies/astr.owl#pi...,16
5,www.astronomoumenos.com/id/wi8924111,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,www.astronomoumenos.com/ontologies/astr.owl#Wi...
6,www.astronomoumenos.com/id/wi4709446,www.astronomoumenos.com/ontologies/astr.owl#wi...,www.astronomoumenos.com/id/wo2432988
7,_bnwi8680929p,www.astronomoumenos.com/ontologies/astr.owl#im...,_bnwi7319865p
8,www.astronomoumenos.com/id/ms9954123,www.astronomoumenos.com/ontologies/astr.owl#ms...,www.astronomoumenos.com/id/wi5028628
9,_bnwi7229868p,www.astronomoumenos.com/ontologies/astr.owl#im...,_bnwi2146711p


In [445]:
result = g.query(
        """
        PREFIX astr: <www.astronomoumenos.com/ontologies/astr.owl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?l_au ?l_wo WHERE {
            ?wo astr:hasContributor ?au ;
                rdfs:label ?l_wo .
            ?au rdfs:label ?l_au .
            }""")

pd.DataFrame(result.bindings)

,l_au,l_wo
0,Themistius rhetor,Presbeuticus Ad Theodosium Imperatorem (or. 14)
1,Ephraem Graecus,Sermo Alius Compunctorius
2,Macarius/Symeon (pseudo),Opera
3,Iohannes Damascenus,De Recta Sententia Liber
4,Basilius Caesariensis,De Ieiunio 2
5,Iohannes Zacharias Actuarius,Opera
6,Agathias scholasticus,Opera
7,Ptolemaeus Claudius,Apotelesmatica seu Tetrabiblos
8,Malchus historicus,De Legationibus Gentium Ad Romanos
9,Magister Thomas,Opera


In [446]:
result = g.query(
        """
        PREFIX astr: <www.astronomoumenos.com/ontologies/astr.owl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?ms ?l_work1 ?l_work2 WHERE {
            ?work1 ^astr:witnessOf/astr:hasClaim/astr:immediatelyFollowedBy/^astr:hasClaim/astr:witnessOf ?work2 .
            ?work1 rdfs:label ?l_work1 .
            ?work2 rdfs:label ?l_work2 .
            ?ms astr:msContains/astr:witnessOf ?work1 .
            }""")

pd.DataFrame(result.bindings)

,l_work1,l_work2,ms
0,Septem Sapientium Conuiuium,Glossae in Hesiodi Opera et dies,www.astronomoumenos.com/id/ms6990712
1,Septem Sapientium Conuiuium,Glossae in Hesiodi Opera et dies,www.astronomoumenos.com/id/ms3331837
2,Septem Sapientium Conuiuium,Glossae in Hesiodi Opera et dies,www.astronomoumenos.com/id/ms2201483
3,Septem Sapientium Conuiuium,In Hesiodi Theogoniam,www.astronomoumenos.com/id/ms6990712
4,Septem Sapientium Conuiuium,In Hesiodi Theogoniam,www.astronomoumenos.com/id/ms3331837
5,Septem Sapientium Conuiuium,In Hesiodi Theogoniam,www.astronomoumenos.com/id/ms2201483
6,Phaedo,De Regno 1-4 (oratt 1-4),www.astronomoumenos.com/id/ms5158165
7,Phaedo,De Regno 1-4 (oratt 1-4),www.astronomoumenos.com/id/ms9327101
8,Phaedo,De Regno 1-4 (oratt 1-4),www.astronomoumenos.com/id/ms3846062
9,Phaedo,De Regno 1-4 (oratt 1-4),www.astronomoumenos.com/id/ms2087803
